# Feature Engineering

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

### Loading the datasets

In [2]:
df_enrolment = pd.read_parquet('enrolment_cleaned.parquet')
df_demographic = pd.read_parquet('demographic_cleaned.parquet')
df_biometric = pd.read_parquet('biometric_cleaned.parquet')

### Aggregating and merging datasets

In [3]:
# 1. AGGREGATE TO [Date, State, District] LEVEL
# We need one row per district per day to calculate lags/ratios
def aggregate_daily(df, col_name):
    # Ensure date is datetime
    df['date'] = pd.to_datetime(df['date'])
    # Group by Date, State, District
    return df.groupby(['date', 'state', 'district']).size().reset_index(name=col_name)

agg_enrol = aggregate_daily(df_enrolment, 'Enrolment_Count')
agg_demo = aggregate_daily(df_demographic, 'Demo_Update_Count')
agg_bio = aggregate_daily(df_biometric, 'Bio_Update_Count')

# 2. MERGE INTO MASTER DATAFRAME
# Outer join ensures we keep days where only one activity happened (e.g. only updates, no enrolments)
df_master = agg_enrol.merge(agg_demo, on=['date', 'state', 'district'], how='outer') \
                     .merge(agg_bio, on=['date', 'state', 'district'], how='outer')

# Fill NaNs with 0 (No activity that day)
df_master = df_master.fillna(0)

# Sort for Time Series calculations
df_master = df_master.sort_values(['state', 'district', 'date'])

In [4]:
df_master.head(10)

,date,state,district,Enrolment_Count,Demo_Update_Count,Bio_Update_Count
0,2025-03-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0
976,2025-04-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0
1829,2025-05-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,2.0
2681,2025-06-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0
3562,2025-07-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0
4454,2025-09-01,Andaman And Nicobar Islands,Andamans,2.0,4.0,6.0
5375,2025-09-02,Andaman And Nicobar Islands,Andamans,1.0,4.0,5.0
6293,2025-09-03,Andaman And Nicobar Islands,Andamans,0.0,5.0,6.0
7221,2025-09-04,Andaman And Nicobar Islands,Andamans,3.0,3.0,3.0
8145,2025-09-05,Andaman And Nicobar Islands,Andamans,0.0,3.0,6.0


### Time Series Features (Lags & Velocity)
##### To help the model "see" the past
##### Lag: "What happened 7 days ago?"
##### Velocity: "Is activity speeding up or slowing down?"

In [9]:
def create_lag_features(df):
    df_feat = df.copy()
    
    # We must group by District to ensure lags don't bleed between districts
    grouper = df_feat.groupby(['state', 'district'])
    
    # 1. Rolling Averages (Smooth out daily noise)
    # 7-Day Moving Average
    df_feat['Enrol_MA_7'] = grouper['Enrolment_Count'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
    df_feat['Demo_MA_7'] = grouper['Demo_Update_Count'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
    
    # 30-Day Moving Average (Long-term trend)
    df_feat['Enrol_MA_30'] = grouper['Enrolment_Count'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())
    
    # 2. Lag Features (Value exactly 1 week ago)
    df_feat['Enrol_Lag_7'] = grouper['Enrolment_Count'].shift(7)
    
    # 3. Velocity (Growth Rate)
    # (Current MA_7 - Previous MA_7) / Previous MA_7
    
    prev_ma_7 = df_feat['Enrol_MA_7'].shift(1)
    
    # Define Epsilon to prevent division by zero (0.000001)
    epsilon = 1e-6
    
    df_feat['Enrol_Velocity'] = (df_feat['Enrol_MA_7'] - prev_ma_7) / (prev_ma_7 + epsilon)
    
    # Fill NaN values generated by shifting (first 7-30 days will be NaN)
    df_feat = df_feat.fillna(0)

    # Capping extreme velocity values to keep the model stable.
    df_feat['Enrol_Velocity'] = df_feat['Enrol_Velocity'].clip(lower=-10, upper=10)
    
    return df_feat

df_features = create_lag_features(df_master)
df_features.head(10)

,date,state,district,Enrolment_Count,Demo_Update_Count,Bio_Update_Count,Enrol_MA_7,Demo_MA_7,Enrol_MA_30,Enrol_Lag_7,Enrol_Velocity
0,2025-03-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000
976,2025-04-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000
1829,2025-05-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,2.0,0.000000,0.000000,0.000000,0.0,0.000000
2681,2025-06-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000
3562,2025-07-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000
4454,2025-09-01,Andaman And Nicobar Islands,Andamans,2.0,4.0,6.0,0.333333,0.666667,0.333333,0.0,10.000000
5375,2025-09-02,Andaman And Nicobar Islands,Andamans,1.0,4.0,5.0,0.428571,1.142857,0.428571,0.0,0.285713
6293,2025-09-03,Andaman And Nicobar Islands,Andamans,0.0,5.0,6.0,0.428571,1.857143,0.375000,0.0,0.000000
7221,2025-09-04,Andaman And Nicobar Islands,Andamans,3.0,3.0,3.0,0.857143,2.285714,0.666667,0.0,0.999998
8145,2025-09-05,Andaman And Nicobar Islands,Andamans,0.0,3.0,6.0,0.857143,2.714286,0.600000,0.0,0.000000


### Interaction Ratios: Features that describe the nature of the activity

In [11]:
# Avoid division by zero
epsilon = 1e-6

# Ratio: Total Updates / New Enrolments
df_features['Update_to_Enrol_Ratio'] = (
    (df_features['Demo_Update_Count'] + df_features['Bio_Update_Count']) / 
    (df_features['Enrolment_Count'] + epsilon)
)

# High Maintenance Day
df_features['Is_High_Maintenance'] = (
    (df_features['Demo_Update_Count'] + df_features['Bio_Update_Count']) > df_features['Enrolment_Count']
).astype(int)

### Geographic Clustering

In [16]:
def add_district_clusters(df):
    # 1. Create a District-Level Summary
    district_summary = df.groupby('district')[['Enrolment_Count', 'Demo_Update_Count', 'Bio_Update_Count']].sum()
    
    # 2. Normalize Data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(district_summary)
    
    # 3. Apply K-Means (3 Clusters)
    kmeans = KMeans(n_clusters=3, random_state=42)
    district_summary['Cluster_ID'] = kmeans.fit_predict(scaled_data)
    
    # Map Cluster ID to readable label based on volume
    cluster_means = district_summary.groupby('Cluster_ID')['Enrolment_Count'].mean().sort_values()
    mapping = {old_label: new_label for new_label, old_label in enumerate(cluster_means.index)}
    district_summary['District_Cluster'] = district_summary['Cluster_ID'].map(mapping)
    
    print("Cluster Centers (Avg Enrolment):")
    print(cluster_means)
    
    # 4. Merge Cluster Label back
    df_merged = df.merge(district_summary[['District_Cluster']].reset_index(), on='district', how='left')
    
    return df_merged

df_final = add_district_clusters(df_features)
df_final.head()

Cluster Centers (Avg Enrolment):
Cluster_ID
2     356.059211
0    1516.146580
1    3427.436782
Name: Enrolment_Count, dtype: float64


,date,state,district,Enrolment_Count,Demo_Update_Count,Bio_Update_Count,Enrol_MA_7,Demo_MA_7,Enrol_MA_30,Enrol_Lag_7,Enrol_Velocity,Update_to_Enrol_Ratio,Is_High_Maintenance,District_Cluster
0,2025-03-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0
1,2025-04-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0
2,2025-05-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2000000.0,1,0
3,2025-06-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0
4,2025-07-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0


### Date and Calendar Features

In [17]:
# Date Components
df_final['Day_OfWeek'] = df_final['date'].dt.dayofweek # 0=Mon, 6=Sun
df_final['Month'] = df_final['date'].dt.month
df_final['Is_Weekend'] = df_final['Day_OfWeek'].isin([5, 6]).astype(int)

# For Quarter End
df_final['Is_Quarter_End'] = df_final['date'].dt.is_quarter_end.astype(int)

In [19]:
df_final.head()

,date,state,district,Enrolment_Count,Demo_Update_Count,Bio_Update_Count,Enrol_MA_7,Demo_MA_7,Enrol_MA_30,Enrol_Lag_7,Enrol_Velocity,Update_to_Enrol_Ratio,Is_High_Maintenance,District_Cluster,Day_OfWeek,Month,Is_Weekend,Is_Quarter_End
0,2025-03-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0,5,3,1,0
1,2025-04-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0,1,4,0,0
2,2025-05-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2000000.0,1,0,3,5,0,0
3,2025-06-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0,6,6,1,0
4,2025-07-01,Andaman And Nicobar Islands,Andamans,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1,0,1,7,0,0


### Saving the Final dataset as a parquet file

In [21]:
# Save to Parquet
output_path = 'final_data/final_modeling_data.parquet'
df_final.to_parquet(output_path)